In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [ ]:
def get_channel_ids(youtube, channel_names):
    """
    Retrieve channel IDs for the given channel names.
    """
    channel_ids = []
    for name in channel_names:
        try:
            request = youtube.search().list(
                part='snippet',
                q=name,
                type='channel',
                maxResults=1
            )
            response = request.execute()
            channel_id = response['items'][0]['id']['channelId']
            channel_ids.append(channel_id)
        except (IndexError, HttpError) as e:
            print(f"Error retrieving channel ID for {name}: {e}")
    return channel_ids

In [ ]:
def get_channel_data(youtube, channel_ids):
    """
    Retrieve subscriber, video, and view counts for the given channel IDs.
    Includes the channel ID in the returned data.
    """
    channel_data = []
    try:
        request = youtube.channels().list(
            part='snippet,statistics',
            id=','.join(channel_ids)
        )
        response = request.execute()
        for item in response.get('items', []):
            channel = {
                'name': item['snippet']['title'],
                'id': item['id'],
                'subscribers': int(item['statistics'].get('subscriberCount', 0)),
                'videos': int(item['statistics'].get('videoCount', 0)),
                'views': int(item['statistics'].get('viewCount', 0)),
            }
            channel_data.append(channel)
    except HttpError as e:
        print(f"Error retrieving channel data: {e}")
    return channel_data

In [ ]:
def get_top_videos(youtube, channel_data, top_n=5):
    """
    Retrieve top N videos sorted by view count for each channel.
    Utilizes the 'search.list' endpoint to fetch top videos and 'videos.list' to get statistics.
    """
    top_videos_data = []
    for channel in channel_data:
        channel_id = channel['id']
        channel_name = channel['name']
        try:
            # Use search.list to get top video IDs sorted by view count
            search_request = youtube.search().list(
                part='id',
                channelId=channel_id,
                maxResults=top_n,
                order='viewCount',
                type='video'
            )
            search_response = search_request.execute()
            video_ids = [item['id']['videoId'] for item in search_response.get('items', [])]
            
            if not video_ids:
                print(f"No videos found for channel '{channel_name}'.")
                continue
            
            # Use videos.list to get statistics for these video IDs
            videos_request = youtube.videos().list(
                part='snippet,statistics',
                id=','.join(video_ids),
                maxResults=top_n
            )
            videos_response = videos_request.execute()
            
            for item in videos_response.get('items', []):
                video = {
                    'Channel Name': channel_name,
                    'Video Title': item['snippet']['title'],
                    'Views': int(item['statistics'].get('viewCount', 0)),
                    'Likes': int(item['statistics'].get('likeCount', 0)),
                    'Comments': int(item['statistics'].get('commentCount', 0))
                }
                top_videos_data.append(video)
                
        except HttpError as e:
            print(f"Error retrieving top videos for channel '{channel_name}': {e}")
    
    return top_videos_data

In [ ]:
# Load the YouTube API key from the .env file
load_dotenv()
API_KEY = os.getenv('YOUTUBE_API_KEY')
if not API_KEY:
    raise ValueError("Please set the YOUTUBE_API_KEY in your .env file")
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [ ]:
channel_names = []
while True:
    # Prompt the user to enter a channel name
    channel_name = input("Enter a YouTube channel name (or '0' to finish): ")
    
    # If the user enters '0', stop asking for channel names
    if channel_name == '0':
        break
    channel_names.append(channel_name)

if not channel_names:
    raise ValueError("No channel names provided.")

In [ ]:
# Retrieve channel IDs
channel_ids = get_channel_ids(youtube, channel_names)
if not channel_ids:
    print("No valid channel IDs retrieved. Exiting.")
    exit()

In [ ]:
# Retrieve channel data
channel_data = get_channel_data(youtube, channel_ids)
if not channel_data:
    print("No channel data available. Exiting.")
    exit()

In [ ]:
# Display channel data
channel_df = pd.DataFrame(channel_data)
print("\nChannel Data:")
print(channel_df[['name', 'subscribers', 'videos', 'views']].to_string(index=False))

In [ ]:
# Retrieve top videos for each channel
top_videos_data = get_top_videos(youtube, channel_data, top_n=5)

# Check if any video data was retrieved
if top_videos_data:
    top_videos_df = pd.DataFrame(top_videos_data)
    # Optionally, sort the DataFrame by Channel Name and Views
    top_videos_df = top_videos_df.sort_values(['Channel Name', 'Views'], ascending=[True, False])
    print("\nTop 5 Videos for Each Channel:")
    print(top_videos_df.to_string(index=False))
else:
    print("\nNo video data available.")